In [ ]:
from keras.applications import vgg16
img_rows, img_cols = 32, 32
VGG=vgg16.VGG16(weights='imagenet',include_top=False ,input_shape=(img_rows, img_cols,3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in VGG.layers:
    layer.trainable=False

In [ ]:
def top(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(128,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import batch_normalization
from tensorflow.keras.models import Model

num_classes = 5

FC_Head = top(VGG, num_classes)

model = Model(inputs = VGG.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/MyDrive/Data Set/train'
validation_data_dir = '/content/drive/MyDrive/Data Set/validation'
 
train_datagen = ImageDataGenerator(rescale=1./255)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 280 images belonging to 5 classes.
Found 274 images belonging to 5 classes.


In [ ]:
from keras.optimizers import rmsprop_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping
                  
checkpoint = ModelCheckpoint("face.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop_v2.RMSprop(learning_rate=0.001),
              metrics = ['accuracy'])

nb_train_samples = 280
nb_validation_samples = 274
 
epochs = 10
batch_size = 32

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)




Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 1.7402 - accuracy: 0.2137
Epoch 1: val_loss improved from inf to 1.57337, saving model to face.h5
8/8 [==============================] - 146s 19s/step - loss: 1.7402 - accuracy: 0.2137 - val_loss: 1.5734 - val_accuracy: 0.2578
Epoch 2/10
8/8 [==============================] - ETA: 0s - loss: 1.5552 - accuracy: 0.3065
Epoch 2: val_loss improved from 1.57337 to 1.37634, saving model to face.h5
8/8 [==============================] - 8s 1s/step - loss: 1.5552 - accuracy: 0.3065 - val_loss: 1.3763 - val_accuracy: 0.5117
Epoch 3/10
8/8 [==============================] - ETA: 0s - loss: 1.3631 - accuracy: 0.4073
Epoch 3: val_loss improved from 1.37634 to 1.13168, saving model to face.h5
8/8 [==============================] - 8s 1s/step - loss: 1.3631 - accuracy: 0.4073 - val_loss: 1.1317 - val_accuracy: 0.5352
Epoch 4/10
8/8 [==============================] - ETA: 0s - loss: 1.5928 - accuracy: 0.4960
Epoch 4: val_loss improved 